In [2]:
%cd ..

D:\ai\GNN


C:\Users\kkire\anaconda3\envs\dl\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
%ls

 Volume in drive D is DATA
 Volume Serial Number is 4096-85EF

 Directory of D:\ai\GNN

10-06-2024  12:18    <DIR>          .
07-06-2024  23:38    <DIR>          ..
12-06-2024  11:26    <DIR>          experiments
08-06-2024  18:03                 0 main.py
08-06-2024  18:11    <DIR>          models
10-06-2024  11:47             2,470 reqirements.txt
10-06-2024  22:26    <DIR>          utils
               2 File(s)          2,470 bytes
               5 Dir(s)  973,212,078,080 bytes free


In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
import torch
from torch import nn
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import global_mean_pool
from torch_geometric.data import Data, Batch
from torch_geometric.loader import DataLoader

from models.layers import gLayer
from models.model import GNN
from utils.train import get_params, train, eval, run_experiment

import time
from scipy import stats
import numpy as np
import random
import networkx as nx
from tqdm import tqdm
import os

In [6]:
clear = os.system('cls')

In [7]:
def hamiltonian_cycle_dynamic_programming(G):
    """
    Dynamic programming algorithm to decide if a Hamiltonian cycle exists in a graph.
    
    Parameters:
    - G: NetworkX graph
    
    Returns:
    - A Hamiltonian cycle as a list of nodes if it exists, otherwise None
    """
    n = len(G.nodes())
    nodes = list(G.nodes())
    index = {nodes[i]: i for i in range(n)}

    # dp[mask][i] will be True if there is a Hamiltonian path that visits all nodes in mask, ending at node i
    dp = [[False] * n for _ in range(1 << n)]
    dp[1][0] = True  # Starting at node 0

    # Iterate over all subsets of nodes
    for mask in range(1 << n):
        for u in range(n):
            if dp[mask][u]:  # If there's a Hamiltonian path for this subset ending at u
                for v in range(n):
                    if mask & (1 << v) == 0 and G.has_edge(nodes[u], nodes[v]):
                        dp[mask | (1 << v)][v] = True

    # Check if there's a Hamiltonian cycle
    final_mask = (1 << n) - 1  # All nodes are visited
    for u in range(1, n):
        if dp[final_mask][u] and G.has_edge(nodes[u], nodes[0]):
            # Reconstruct the path
            path = [0]
            mask = final_mask
            last = u
            while mask != 1:
                for v in range(n):
                    if mask & (1 << v) and dp[mask ^ (1 << last)][v] and G.has_edge(nodes[v], nodes[last]):
                        path.append(last)
                        mask ^= (1 << last)
                        last = v
                        break
            path.append(0)
            path.reverse()
            return True
    return False

In [8]:
def create_hamiltonian_cycle_graph(n):
    """
    Creates a graph with a known Hamiltonian cycle.
    Parameters:
    - n: number of nodes in the graph
    Returns:
    - G: NetworkX graph with a Hamiltonian cycle
    """
    # Create a cycle graph with n nodes
    G = nx.cycle_graph(n)
    
    # Optionally, add some random edges to make the graph more complex
    # Ensure the added edges do not disrupt the Hamiltonian cycle
    
    num_additional_edges = random.randint(0, n * (n - 1) // 2 - n)  # Add additional edges
    
    possible_edges = list(nx.complement(G).edges())
    random.shuffle(possible_edges)
    additional_edges = possible_edges[:num_additional_edges]
    G.add_edges_from(additional_edges)
    
    return G

In [9]:
def create_random_graph_without_hamiltonian_cycle(n1, n2):
    """
    Creates a bipartite graph with n1 and n2 nodes in each partition that does not have a Hamiltonian cycle.
    
    Parameters:
    - n1: number of nodes in the first partition
    - n2: number of nodes in the second partition
    
    Returns:
    - G: NetworkX bipartite graph without a Hamiltonian cycle
    """
    if abs(n1 - n2) != 1:
        raise ValueError("For the bipartite graph to guarantee no Hamiltonian cycle, partitions should differ by 1.")
    
    G = nx.complete_bipartite_graph(n1, n2)
    return G

In [125]:
def create_graph_dataset(num_samples, edge_dim, batch_size, split=(0.7, 0.9)):
    
    data_list = []
    
    for i in tqdm(range(num_samples)):
        n = random.randint(3, 15)
        t = random.random() 
        t2 = random.random()
        
        if t <0.5:
            G = create_hamiltonian_cycle_graph(n)
        else:
            if t2 < 0.5:
                if n%2 == 0:
                    n -= 1
                k1, k2 = (n+1)//2, (n-1)//2
                
                G = create_random_graph_without_hamiltonian_cycle(k1, k2)
            else:
                p = (stats.norm.cdf(random.gauss()))*(0.71)
                G = nx.gnp_random_graph(n, p)
                
        x = torch.zeros((n, 1))
        edge_index = [[], []]
        edge_attr = []
        
        for u, v in G.edges:
            x[u] += 1
            x[v] += 1
            edge_index[0].append(u)
            edge_index[1].append(v)
            edge_attr.append(torch.randn(edge_dim))        
        
        if t < 0.5 or (t>=0.5 and hamiltonian_cycle_dynamic_programming(G)):
            y = 1
        else:
            y = 0

        for i in range(n):
            if x[i] == 0:
                edge_index[0].append(i)
                edge_index[1].append(i)
                edge_attr.append(torch.Tensor(np.random.standard_cauchy(edge_dim)))
                
        edge_index = torch.Tensor(edge_index).type(torch.LongTensor)
        edge_attr = torch.stack(edge_attr)
        
        d = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y)
        data_list.append(d)

    random.shuffle(data_list)

    train_dataset, val_dataset, test_dataset = data_list[:int(num_samples*split[0])], data_list[int(num_samples*split[0]):int(num_samples*split[1])], data_list[int(num_samples*split[1]):]
    
    train_loader, val_loader, test_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True), DataLoader(val_dataset, batch_size=batch_size, shuffle=True), DataLoader(test_dataset, batch_size=batch_size, shuffle=True)     
    
    return train_loader, val_loader, test_loader

In [126]:
def check_dataset_balance(dataloader, n_classes, batch_size):
    
    classes = [0 for _ in range(len(n_classes))]
    
    print("checking skewing of dataset")
    
    for d in tqdm(dataloader) :
        for i in range(min(len(d), batch_size)):
            classes[d[i].y] += 1

    print("the dataset contains:")
    
    for j in range(len(classes)):    
        print(f"{classes[j]} instances of {n_classes[j]}")
    

In [127]:
batch_size=16
edge_dim = 4
num_samples = 10000

class_desc = {0:"hamiltonian cycle", 1:"no hamiltonian cycle"}

print(f"creating dataset containing {num_samples} graphs and batch size of {batch_size}")
train_loader, val_loader, test_loader = create_graph_dataset(num_samples, edge_dim, batch_size)


creating dataset containing 10000 graphs and batch size of 16


100%|████████████████| 10000/10000 [04:33<00:00, 36.60it/s]


In [128]:
# print("checking the balance of classes in dataset")
check_dataset_balance(train_loader, class_desc, batch_size)

checking skewing of dataset


100%|████████████████████| 438/438 [00:04<00:00, 90.03it/s]

the dataset contains:
2870 instances of hamiltonian cycle
4130 instances of no hamiltonian cycle


In [ ]:
'''
BIG PROBLEM

GNNs by torch_geometric uses the scatter function which calculates the messages given by using the edge_index[1] but this causes problem as edge_index might not contain some edges does to isolated nodes
added self loops to isolated edges and the edge attr of these self loops is sampled from standard_cauchy which is highly different from normal distribution

'''

In [120]:
del sanity_model
sanity_model = GNN(4, 50, 4, 1, 2)
print(sanity_model)

GNN(
  (lin_in): Linear(in_features=1, out_features=50, bias=True)
  (convs): ModuleList(
    (0-3): 4 x gLayer(emb_dim=50, aggr=add)
  )
  (lin_pred): Linear(in_features=50, out_features=2, bias=True)
)


In [121]:
ll = nn.CrossEntropyLoss()

In [124]:

with torch.no_grad():
    
    for d in val_loader:
        # dd = d[0]
        # print(dd)
        y_pr = sanity_model(d)
        # print(dd)
        # print(y_pr)
        
        # print(dd.y)
        print(ll(y_pr, d.y))
           
            

b torch.Size([38, 50]) torch.Size([2, 101]) torch.Size([101, 4])
tensor(5.0383)
b torch.Size([37, 50]) torch.Size([2, 35]) torch.Size([35, 4])
tensor(0.9864)
b torch.Size([31, 50]) torch.Size([2, 54]) torch.Size([54, 4])
tensor(1.4234)
b torch.Size([43, 50]) torch.Size([2, 117]) torch.Size([117, 4])
tensor(2.5806)
b torch.Size([26, 50]) torch.Size([2, 52]) torch.Size([52, 4])
tensor(3.8618)


In [141]:
n_layers = 4
emb_dim = 50
in_dim = 1    # in_dim = 1 as the nodes are portraied as (n, 1) the embedding describes the degree of that node 
out_dim = len(class_desc)
epochs = 100

model = GNN(n_layers=n_layers, emb_dim=emb_dim, in_dim=in_dim, out_dim=out_dim)
print(type(model).__name__)
print()
print(model)
print(f'Total parameters: {get_params(model)}')

loss_func = nn.CrossEntropyLoss() 

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# LR scheduler which decays LR when validation metric doesn't improve
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.9, patience=5, min_lr=0.00001)

model_name = type(model).__name__

best_val_error, test_error, train_time, perf_per_epoch = run_experiment(
    model, 
    model_name, 
    epochs,
    loss_func,
    optimizer,
    scheduler,
    train_loader,
    val_loader, 
    test_loader,
)

# For storing experimental results over the course of the practical
RESULTS = {}
DF_RESULTS = pd.DataFrame(columns=["Loss", "Test MAE", "Val MAE", "Epoch", "Model"])

RESULTS[model_name] = (best_val_error, test_error, train_time)
df_temp = pd.DataFrame(perf_per_epoch, columns=["Loss", "Test MAE", "Val MAE", "Epoch", "Model"])
DF_RESULTS = DF_RESULTS.append(df_temp, ignore_index=True)

GNN

GNN(
  (lin_in): Linear(in_features=1, out_features=50, bias=True)
  (convs): ModuleList(
    (0-3): 4 x gLayer(emb_dim=50, aggr=add)
  )
  (lin_pred): Linear(in_features=50, out_features=2, bias=True)
)
Total parameters: 63402
Running experiment for GNN, training on 7000 samples for 100 epochs.

Model architecture:
GNN(
  (lin_in): Linear(in_features=1, out_features=50, bias=True)
  (convs): ModuleList(
    (0-3): 4 x gLayer(emb_dim=50, aggr=add)
  )
  (lin_pred): Linear(in_features=50, out_features=2, bias=True)
)
Total parameters: 63402

Start training:


  1%|▏                   | 1/100 [00:57<1:34:25, 57.23s/it]


KeyboardInterrupt: 